# RNA localization script

### Library imports

In [ ]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "True"

from cellpose import models
import bigfish.stack as stack
import torch
import numpy as np
import bigfish.detection as detection
import bigfish.multistack as multistack
import bigfish.plot as plot
import bigfish.segmentation as segmentation
import pandas as pd 

### Local paths

TIF image must be written without the ".TIF" extension. It should be located in the input folder.

In [ ]:
path_input = r"C:\Users\thoma\data\Data Adham\Modified script\input"
path_output = r"C:\Users\thoma\data\Data Adham\Modified script\output"

tif_file_name = "1_TOP2A smFISH 2_p-bodies 3_DAPI"

Load image

In [ ]:
path = os.path.join(path_input, f"{tif_file_name}.TIF")
image = stack.read_image(path)

Read different channels

In [ ]:
# To avoid using a too big image, we crop it
PICTURE_MAX_SIZE = 1000

# Here one may change the channels order depending on the data
rna1 = np.copy(image[:, :, 0])
rna1 = rna1[0:PICTURE_MAX_SIZE,0:PICTURE_MAX_SIZE]
print("smfish channel")
print("\r shape: {0}".format(rna1.shape))
print("\r dtype: {0}".format(rna1.dtype))
# Note that here we use only one RNA channel, but you can use several by changing following line
rnas = [rna1]

gfp = np.copy(image[:, :, 1])
gfp = gfp[0:PICTURE_MAX_SIZE,0:PICTURE_MAX_SIZE]
print("GFP channel")
print("\r shape: {0}".format(gfp.shape))
print("\r dtype: {0}".format(gfp.dtype))

nuc = np.copy(image[:, :, 2])
nuc = nuc[0:PICTURE_MAX_SIZE,0:PICTURE_MAX_SIZE]
print("DAPI channel")
print("\r shape: {0}".format(nuc.shape))
print("\r dtype: {0}".format(nuc.dtype))

Plot loaded images

In [ ]:
plot.plot_images([*rnas, gfp, nuc], contrast=True)

## 1 - Nucleus and cell segmentation using Cellpose & Bigfish

Parameters

In [ ]:
# Typical nucleus diameter, in pixels
NUCLEUS_DIAMETER = 150 

# Minimum intensity on the RNA channel to be considered as cell instead of background
RNA_BACKGROUND_THRESHOLD = 400

Run Cellpose nucleus segmentation

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.Cellpose(model_type="nuclei", device=device)
results, _, _, _ = model.eval([nuc], channels=[0, 0], diameter=NUCLEUS_DIAMETER)

# Create binary mask form Cellpose segmentation
nuc_mask = np.copy(results[0])
nuc_mask[nuc_mask > 0] = 1

#Labeling
nuc_label = np.copy(results[0])
nuc_label = np.int64(nuc_label)
print("nuclei labels")
print("\r shape: {0}".format(nuc_label.shape))
print("\r dtype: {0}".format(nuc_label.dtype))

Plot nucleus segmentation results

In [ ]:
plot.plot_segmentation(nuc, nuc_mask, rescale=True, framesize=(15, 5))
print("nuclei mask")
print("\r shape: {0}".format(nuc_mask.shape))
print("\r dtype: {0}".format(nuc_mask.dtype))

Perform cell segmentation using Bigfish watershed & save result in output folder

In [ ]:
cell_label = segmentation.cell_watershed(rnas[0], nuc_label, threshold=RNA_BACKGROUND_THRESHOLD, alpha=0.9)
print("cells labels")
print("\r shape: {0}".format(cell_label.shape))
print("\r dtype: {0}".format(cell_label.dtype))


segmentation_path = os.path.join(path_output, f"{tif_file_name}_segmentation")
plot.plot_segmentation_boundary(rnas[0], cell_label, nuc_label, framesize=(25, 25), contrast=True, path_output=segmentation_path, show=False)
cell_mask = segmentation.thresholding(rnas[0], threshold=RNA_BACKGROUND_THRESHOLD)

Plot cell segmentation results

In [ ]:
plot.plot_images([cell_mask, cell_label], 
                 titles=["Binary mask", "Labelled cells"], 
                 framesize=(10, 10))

## 2 - RNA molecules counting

Improve RNA and GFP contrast

In [ ]:
rnas_contrasted = []
for rna_image in rnas:
    rna_contrasted = stack.rescale(rna_image, channel_to_stretch=0)
    rnas_contrasted.append(rna_contrasted)

gfp_contrasted = stack.rescale(gfp, channel_to_stretch=0)


Detect RNA spots and plot results

In [ ]:
spots_rnas = []

for rna_image in rnas:
    spots_rna, threshold = detection.detect_spots(
        images=rna_image,
        threshold=None,
        remove_duplicate=True,
        return_threshold=True,
        voxel_size=72,
        spot_radius=100)
    # Plot detection results
    plot.plot_detection(
        image=rna_image,
        spots=spots_rna,
        radius=0.5,
        fill=True,
        contrast=True,
        framesize=(20, 15))
    spots_rnas.append(spots_rna)

Decompose RNA clusters and plot results

In [ ]:
spots_decomposed_rnas = []
for spots_rna, rna_image in zip(spots_rnas, rnas):
    spots_decomposed_rna, _, _ = detection.decompose_dense(
        alpha=0.45, 
        beta=1, 
        gamma=5, 
        image=rna_image,
        spots=spots_rna,
        voxel_size=72,
        spot_radius=100)
    print("Detected spots before decomposition")
    print(spots_decomposed_rna.shape, spots_decomposed_rna.dtype)
    print("Detected spots after decomposition")
    print(spots_decomposed_rna.shape, spots_decomposed_rna.dtype)
    plot.plot_detection(
        image=rna_image,
        spots=spots_decomposed_rna,
        radius=0.5,
        fill=True,
        contrast=True,
        framesize=(25, 20))
    spots_decomposed_rnas.append(spots_decomposed_rna)

Remove of out-of-cell spots and plot results

In [ ]:
spots_in_cell_rnas = []

for spots_decomposed_rna, rna_image in zip(spots_decomposed_rnas, rnas):
    index = 0
    spots_in_cell_rna = spots_decomposed_rna
    for spot in spots_in_cell_rna:
        y = spot[0]
        x = spot[1]
        if cell_label[y, x] == 0:
            spots_in_cell_rna = np.delete(spots_in_cell_rna,index,0)
        else:
            index += 1
    print("detected spots everywhere")
    print("\r shape: {0}".format(spots_decomposed_rna.shape))
    print("detected RNAs inside cells")
    print("\r shape: {0}".format(spots_in_cell_rna.shape))

    plot.plot_detection(rna_image, spots_in_cell_rna,contrast=True, radius=0.5)
    spots_in_cell_rnas.append(spots_in_cell_rna)

## 3 - Recruited RNAs counting

Parameters

In [ ]:
# Minimum intensity on the gfp channel to be considered as cell instead of background
GFP_THRESHOLD = 2100

Create GFP binary mask

In [ ]:
gfp_mask = segmentation.thresholding(gfp, GFP_THRESHOLD)
gfp_mask = stack.median_filter(gfp_mask.astype(np.uint8), "disk", 3).astype(bool)
gfp_mask = stack.dilation_filter(gfp_mask, "disk", 1).astype(bool)

Plot segmentation results

In [ ]:
plot.plot_segmentation(gfp, gfp_mask, rescale=True, title="dilated mask", framesize=(20, 20))

Get recruited RNAs for each detected cell

In [ ]:
rnas_recruited = []
for i, spots_in_cell_rna in enumerate(spots_in_cell_rnas):
    index = 0
    rna_recruited = spots_in_cell_rna
    for rna in spots_in_cell_rna:
        y = rna[0]
        x = rna[1]
        if not gfp_mask[y, x]:
            rna_recruited = np.delete(rna_recruited,index,0)
        else:
            index += 1
    print(f"RNA{i+1} molecules in cells")
    print("\r shape: {0}".format(spots_in_cell_rna.shape))
    print(f"RNA{i+1} molecules in P-bodies")
    print("\r shape: {0}".format(rna_recruited.shape))
    ratio = len(rna_recruited) / len (spots_in_cell_rna) *100
    print("P-body localized RNAs{0}: {1} ({2:0.3}%)".format(i+1, len(rna_recruited), ratio))
    plot.plot_detection(gfp, rna_recruited,contrast=True, radius=0.5)
    rnas_recruited.append(rna_recruited)

## 4 - Results summary

Extract results for each field-of-view

In [ ]:
fov_results_rnas = []
for spots_in_cell_rna, rna_recruited, rna_contrasted in zip(spots_in_cell_rnas, rnas_recruited, rnas_contrasted):
    fov_results_rna = multistack.extract_cell(
        cell_label=cell_label, 
        ndim=2, 
        nuc_label=nuc_label, 
        rna_coord =spots_in_cell_rna,
        others_coord={"rna_recruited": rna_recruited},
        image=rna_contrasted,
        others_image={"dapi": nuc, "smfish": rna_contrasted, "gfp" : gfp_contrasted},
        remove_cropped_cell=False)
    fov_results_rnas.append(fov_results_rna)
print("Number of cells identified: {0}".format(len(fov_results_rna)))

Save results as images

In [ ]:
for j, fov_results_rna in enumerate(fov_results_rnas):
    for i, cell_results in enumerate(fov_results_rna):
        print("cell {0}".format(i))
        
        # Get cell results
        cell_mask = cell_results["cell_mask"]
        cell_coord = cell_results["cell_coord"]
        nuc_coord = cell_results["nuc_coord"]
        nuc_mask = cell_results["nuc_mask"]
        rna_coord = cell_results["rna_coord"]
        rna_recruit_coord = cell_results["rna_recruited"]
        image_contrasted = cell_results["image"]
        gfp_contrasted = cell_results["gfp"]
        print("\r number of rna {0}".format(len(rna_coord)))
        print("\r number of rna recruited {0}".format(len(rna_recruit_coord)))
        if len(rna_coord) != 0:
            ratio = len(rna_recruit_coord) / len (rna_coord) *100
        print("recruited RNAs: {0} ({1:0.3}%)".format(len(rna_recruit_coord), ratio))
        
        # Plot cells
        cell_path = os.path.join(path_output, f"{tif_file_name}_segmentation_cell_{i}")
        plot.plot_cell(
            ndim=3,  
            image=image_contrasted, 
            cell_mask=cell_mask, 
            nuc_mask=nuc_mask, 
            title="Cell {0}".format(i), 
            rescale=True, 
            framesize=(10, 10),
            path_output = cell_path,
            show=False)
        rna_coord_path = os.path.join(path_output, f"{tif_file_name}_all_RNA{j+1}_cell_{i}")
        plot.plot_detection(image_contrasted, rna_coord,contrast=True,framesize=(15, 15), radius=0.5, path_output=rna_coord_path, show=False)
        rna_recruit_coord_path = os.path.join(path_output, f"{tif_file_name}_recruited_RNA{j+1}_cell_{i}")
        plot.plot_detection(gfp_contrasted, rna_recruit_coord,contrast=True, framesize=(15, 15), radius=0.5, path_output=rna_recruit_coord_path, show=False)


Merge results and display first elements in table

In [ ]:
data_frames = [] 
for i, fov_results_rna in enumerate(fov_results_rnas):
    df = multistack.summarize_extraction_results(fov_results_rna, ndim=2)
    df.rename(columns=lambda x: x.replace("rna", f"rna{i+1}"), inplace=True)
    data_frames.append(df)

# Merge data frames if more than 1 RNA channel
if  len(data_frames) == 1:
    df = data_frames[0] 
else:
    df = None
    for i in range(1, len(data_frames)):
        cols_to_use = data_frames[i].columns.difference(data_frames[0].columns)
        if df is None:
            df = pd.merge(data_frames[0], data_frames[i][cols_to_use], left_index=True, right_index=True, how='outer')
        else:
            df = pd.merge(df, data_frames[i][cols_to_use], left_index=True, right_index=True, how='outer')

print("shape: {0}".format(df.shape))
df.head(100)

Save results in a .csv file

In [ ]:
csv_path = os.path.join(path_output, f"{tif_file_name}_RNA.csv")
df.to_csv(csv_path)